In [88]:
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.7.1
False


In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [12]:
import os
import cv2
import shutil
import json
import numpy as np

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
with open('../data/iSAID/image_by_category.json', 'r') as fp:
    image_by_category = json.load(fp)
with open('../data/iSAID/idx_by_category.json', 'r') as fp:
    idx_by_category = json.load(fp)
with open('../data/iSAID/category_id2name.json', 'r') as fp:
    category_id2name = json.load(fp)

In [9]:
image_by_category['7'][:3]

['../data/iSAID/val/images/P0003.png',
 '../data/iSAID/val/images/P0004.png',
 '../data/iSAID/val/images/P0007.png']

In [10]:
img_path = '../data/iSAID/val/images'
anno_path = '../data/iSAID/val/iSAID_val.json'
extra_meta = {}     # no additional metadata to be associated w/ dataset

In [11]:
register_coco_instances('isaid_full', extra_meta, anno_path, img_path)

In [13]:
data_list = DatasetCatalog.get('isaid_full')
meta_list = MetadataCatalog.get('isaid_full')

[03/30 20:03:40 d2.data.datasets.coco]: Loading ../data/iSAID/val/iSAID_val.json takes 13.50 seconds.
[03/30 20:03:41 d2.data.datasets.coco]: Loaded 458 images in COCO format from ../data/iSAID/val/iSAID_val.json


In [14]:
meta_list

namespace(name='isaid_full',
          json_file='../data/iSAID/val/iSAID_val.json',
          image_root='../data/iSAID/val/images',
          evaluator_type='coco',
          thing_classes=['ship',
                         'storage_tank',
                         'baseball_diamond',
                         'tennis_court',
                         'basketball_court',
                         'Ground_Track_Field',
                         'Bridge',
                         'Large_Vehicle',
                         'Small_Vehicle',
                         'Helicopter',
                         'Swimming_pool',
                         'Roundabout',
                         'Soccer_ball_field',
                         'plane',
                         'Harbor'],
          thing_dataset_id_to_contiguous_id={1: 0,
                                             2: 1,
                                             3: 2,
                                             4: 3,
                      

In [15]:
# create a separate data with only commercial trucks

In [25]:
truck_files_filtered_list = []
for idx in idx_by_category['7']:
    d = data_list[idx]
    # Large image files will cause inconveniences for this demo. Skipping.
    # TODO for large image files, split image and mask into smaller parts
    if max(d['height'], d['width']) > 2000: 
        print(f"skipping large image: {d['file_name']}")
    else:
        truck_files_filtered_list.append(d['file_name'])

skipping large image: ../data/iSAID/val/images/P0007.png
skipping large image: ../data/iSAID/val/images/P0019.png
skipping large image: ../data/iSAID/val/images/P0027.png
skipping large image: ../data/iSAID/val/images/P0047.png
skipping large image: ../data/iSAID/val/images/P0060.png
skipping large image: ../data/iSAID/val/images/P0128.png
skipping large image: ../data/iSAID/val/images/P0146.png
skipping large image: ../data/iSAID/val/images/P0161.png
skipping large image: ../data/iSAID/val/images/P0168.png
skipping large image: ../data/iSAID/val/images/P0179.png
skipping large image: ../data/iSAID/val/images/P0491.png
skipping large image: ../data/iSAID/val/images/P0589.png
skipping large image: ../data/iSAID/val/images/P0683.png
skipping large image: ../data/iSAID/val/images/P0696.png
skipping large image: ../data/iSAID/val/images/P0704.png
skipping large image: ../data/iSAID/val/images/P0795.png
skipping large image: ../data/iSAID/val/images/P0837.png
skipping large image: ../data/i

In [26]:
len(truck_files_filtered_list)

109

In [27]:
truck_files_filtered_array = np.array(truck_files_filtered_list)
np.random.shuffle(truck_files_filtered_array)

In [29]:
# 70/30 split
train_size = int(len(truck_files_filtered_array) * .7)
train_files = truck_files_filtered_array[:train_size]
val_files = truck_files_filtered_array[train_size:]
print(f"Training set {len(train_files)} \t Validation set {len(val_files)}")

Training set 76 	 Validation set 33


In [30]:
truck_img_path = '../data/iSAID/val/truck'
truck_img_train_path = os.path.join(truck_img_path, 'train')
truck_img_val_path = os.path.join(truck_img_path, 'val')
os.makedirs(truck_img_train_path, exist_ok=True)
os.makedirs(truck_img_val_path, exist_ok=True)
for src in train_files:
    shutil.copyfile(src, os.path.join(truck_img_train_path, os.path.basename(src)))
for src in val_files:
    shutil.copyfile(src, os.path.join(truck_img_val_path, os.path.basename(src)))

In [36]:
# we also need to subset the json
with open(anno_path, 'r') as fp:
    full_anno = json.load(fp)

In [37]:
full_anno.keys()

dict_keys(['images', 'categories', 'annotations'])

In [39]:
full_anno['images'][0]

{'id': 0,
 'width': 1147,
 'height': 1023,
 'file_name': 'P0003.png',
 'ins_file_name': 'P0003_instance_id_RGB.png',
 'seg_file_name': 'P0003_instance_color_RGB.png'}

In [41]:
full_anno['categories']

[{'id': 2, 'name': 'storage_tank'},
 {'id': 8, 'name': 'Large_Vehicle'},
 {'id': 9, 'name': 'Small_Vehicle'},
 {'id': 1, 'name': 'ship'},
 {'id': 15, 'name': 'Harbor'},
 {'id': 3, 'name': 'baseball_diamond'},
 {'id': 6, 'name': 'Ground_Track_Field'},
 {'id': 13, 'name': 'Soccer_ball_field'},
 {'id': 11, 'name': 'Swimming_pool'},
 {'id': 12, 'name': 'Roundabout'},
 {'id': 4, 'name': 'tennis_court'},
 {'id': 5, 'name': 'basketball_court'},
 {'id': 14, 'name': 'plane'},
 {'id': 10, 'name': 'Helicopter'},
 {'id': 7, 'name': 'Bridge'}]

In [45]:
len(full_anno['annotations'])

116649

In [71]:
def subset_full_anno(full_anno, subset_filenames, category_name='Large_Vehicle'):
    out = {'images':[], 'categories':[], 'annotations':[]}
    subset_filenames = set(subset_filenames)  # faster lookup
    out['images'] = [i for i in full_anno['images'] if i['file_name'] in subset_filenames ]
    image_ids = set([i['id'] for i in out['images']])
    out['annotations'] = [a for a in full_anno['annotations'] if a['category_name'] == category_name\
                          and a['image_id'] in image_ids]
    out['categories'] = [{'id': 8, 'name': 'Large_Vehicle'}]
    return out

In [72]:
train_anno = subset_full_anno(full_anno, [os.path.basename(fp) for fp in train_files])
val_anno = subset_full_anno(full_anno, [os.path.basename(fp) for fp in val_files])

In [73]:
with open('../data/iSAID/val/truck/train_anno.json', 'w') as fp:
    json.dump(train_anno, fp)
with open('../data/iSAID/val/truck/val_anno.json', 'w') as fp:
    json.dump(val_anno, fp)

In [74]:
register_coco_instances('isaid_truck_train3', 
                        extra_meta, 
                        '../data/iSAID/val/truck/train_anno.json',
                        truck_img_train_path)
register_coco_instances('isaid_truck_val3', 
                        extra_meta, 
                        '../data/iSAID/val/truck/val_anno.json',
                        truck_img_val_path)

In [99]:
truck_img_train_path

'../data/iSAID/val/truck/train'

In [75]:
# sanity check
data_list = DatasetCatalog.get('isaid_truck_train3')
meta_list = MetadataCatalog.get('isaid_truck_train3')

WARNING [03/30 21:17:29 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/30 21:17:29 d2.data.datasets.coco]: Loaded 76 images in COCO format from ../data/iSAID/val/truck/train_anno.json


In [76]:
meta_list

namespace(name='isaid_truck_train3',
          json_file='../data/iSAID/val/truck/train_anno.json',
          image_root='../data/iSAID/val/truck/train',
          evaluator_type='coco',
          thing_classes=['Large_Vehicle'],
          thing_dataset_id_to_contiguous_id={8: 0})

In [78]:
data_list = DatasetCatalog.get('isaid_truck_val3')
meta_list = MetadataCatalog.get('isaid_truck_val3')

WARNING [03/30 21:19:04 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/30 21:19:04 d2.data.datasets.coco]: Loaded 33 images in COCO format from ../data/iSAID/val/truck/val_anno.json


In [79]:
meta_list

namespace(name='isaid_truck_val3',
          json_file='../data/iSAID/val/truck/val_anno.json',
          image_root='../data/iSAID/val/truck/val',
          evaluator_type='coco',
          thing_classes=['Large_Vehicle'],
          thing_dataset_id_to_contiguous_id={8: 0})